In [2]:
from datasets import load_dataset
import pandas as pd
import numpy as np
import re
import json

from unicode import split_syllables, join_jamos
from tqdm import tqdm

## Data Load

In [3]:
text = "text"

In [4]:
all_data = load_dataset('csv',data_files='./order_speech_ko.csv',split='train')

Using custom data configuration default-8e0d0b8910acc620
Reusing dataset csv (C:\Users\Mu-jun\.cache\huggingface\datasets\csv\default-8e0d0b8910acc620\0.0.0\bf68a4c4aefa545d0712b2fcbb1b327f905bbe2f6425fbc5e8c25234acb9e14a)


## Data Cleaning

In [5]:
chars_to_ignore_regex = '[\,\?\.\!\-\;\:\"\“\%\‘\”\�]'

def remove_special_characters(batch):
    batch[text] = re.sub(chars_to_ignore_regex, '', batch[text]).lower() + " "
    return batch

In [6]:
remove_spectial_char_data = all_data.map(remove_special_characters)

Parameter 'function'=<function remove_special_characters at 0x000002022A1B5F78> of the transform datasets.arrow_dataset.Dataset._map_single couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


  0%|          | 0/142367 [00:00<?, ?ex/s]

## Split to Font

In [7]:
target_data = remove_spectial_char_data
jamos = []
for i in tqdm(range(len(target_data))):
    split_text = split_syllables(target_data[i][text])
    all_text = " ".join(split_text)
    jamos.extend(list(set(all_text)))

100%|██████████| 142367/142367 [00:25<00:00, 5527.54it/s]


## Add index

In [8]:
vocab_list = list(set(jamos))

In [9]:
vocab_dict = {v: k+5 for k, v in enumerate(vocab_list)}

## Add Special Token

In [12]:
vocab_dict["|"] = 4
vocab_dict["<pad>"] = 0
vocab_dict["<s>"] = 1
vocab_dict["</s>"] = 2
vocab_dict["<unk>"] = 3
len(vocab_dict)

59

## Export Vocabulary to json file

In [13]:
with open('vocab.json', 'w') as vocab_file:
    json.dump(vocab_dict, vocab_file)